# Getting started

## Before you start

In this notebook we give you a short introduction into the workings of promptolution.

We will use the OpenAI-API to demonstrate the functionality of promptolution, however we also provide a local LLM, as well as a vLLM backend. You can also change the `base_url` in the config, in order to use any other api, that follows the OpenAI API standard.

Thanks for giving it a try!

## Installs

In [41]:
# ! pip install promptolution

## Imports

In [43]:
import pandas as pd
from promptolution import ExperimentConfig, run_experiment
import nest_asyncio
nest_asyncio.apply() # we need this only because we are in a notebook

## set up llms, predictor, tasks and optimizer

Here we set up our dataset. We use the subjectivity dataset from hugging face, but of course here you may want to use your own dataset.

Just make sure, to name the input column "x" and the target column "y", as well as providing a short dataset description.

In [44]:
df = pd.read_csv("hf://datasets/tasksource/subjectivity/train.csv")
df = df.rename(columns={"Sentence": "x", "Label": "y"})
df = df.replace({"OBJ": "objective", "SUBJ": "subjective"})

task_description = "The dataset contains sentences labeled as either subjective or objective. "\
        "The task is to classify each sentence as either subjective or objective. " \
        "The class mentioned first in the response of the LLM will be the prediction."

We definied some initial prompts, however you may also take a look at `create_prompts_from_samples` in order to automatically generate them.

In [45]:
init_prompts = [
    'Classify the given text as either an objective or subjective statement based on the tone and language used: e.g. the tone and language used should indicate whether the statement is a neutral, factual summary (objective) or an expression of opinion or emotional tone (subjective). Include the output classes "objective" or "subjective" in the prompt.',
    'What kind of statement is the following text: [Insert text here]? Is it <objective_statement> or <subjective_statement>?',
    'Identify whether a sentence is objective or subjective by analyzing the tone, language, and underlying perspective. Consider the emotion, opinion, and bias present in the sentence. Are the authors presenting objective facts or expressing a personal point of view? The output will be either "objective" (output class: objective) or "subjective" (output class: subjective).',
    'Classify the following sentences as either objective or subjective, indicating the name of the output classes: [input sentence]. Output classes: objective, subjective',
    '_query a text about legal or corporate-related issues, and predict whether the tone is objective or subjective, outputting the corresponding class "objective" for non-subjective language or "subjective" for subjective language_',
    'Classify a statement as either "subjective" or "objective" based on whether it reflects a personal opinion or a verifiable fact. The output classes to include are "objective" and "subjective".',
    'Classify the text as objective or subjective based on its tone and language.',
    'Classify the text as objective or subjective based on the presence of opinions or facts. Output classes: objective, subjective.',
    'Classify the given text as objective or subjective based on its tone, focusing on its intention, purpose, and level of personal opinion or emotional appeal, with outputs including classes such as objective or subjective.',
    "Categorize the text as either objective or subjective, considering whether it presents neutral information or expresses a personal opinion/bias.\n\nObjective: The text has a neutral tone and presents factual information about the actions of Democrats in Congress and the union's negotiations.\n\nSubjective: The text has a evaluative tone and expresses a positive/negative opinion/evaluation about the past performance of the country.",
    'Given a sentence, classify it as either "objective" or "subjective" based on its tone and language, considering the presence of third-person pronouns, neutral language, and opinions. Classify the output as "objective" if the tone is neutral and detached, focusing on facts and data, or as "subjective" if the tone is evaluative, emotive, or biased.',
    'Identify whether the given sentence is subjective or objective, then correspondingly output "objective" or "subjective" in the form of "<output class>, (e.g. "objective"), without quotes. Please note that the subjective orientation typically describes a sentence where the writer expresses their own opinion or attitude, whereas an objective sentence presents facts or information without personal involvement or bias. <output classes: subjective, objective>'
]

We will be now using the gpt

In [46]:
token = open("../deepinfratoken.txt", "r").read()

In [47]:
config = ExperimentConfig(
    task_description=task_description,
    prompts=init_prompts,
    n_steps=3,
    optimizer="evopromptga",
    api_url="https://api.openai.com/v1",
    llm="gpt-4o-mini-2024-07-18",
    token=token,
)

In [48]:
prompts = run_experiment(df, config)

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-3DmWJfR4tphuKTSzcsMB3vHF on requests per min (RPM): Limit 500, Used 500, Requested 1. Please try again in 120ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}

In [ ]:
prompts